## BERT

In [1]:
import pandas as pd
## for data
import json
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, SpatialDropout1D
from keras.layers.recurrent import LSTM
from keras.layers import Activation, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns

## for processing
import re
import nltk
from nltk.tokenize import word_tokenize
## for bag-of-words
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing, feature_selection, metrics

## for explainer
from lime import lime_text

## for word embedding
import gensim
import gensim.downloader as gensim_api

## for deep learning
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K

## for bert language model
import transformers

C:\Users\Dee\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
dtf_new = pd.read_csv("data/data_yoruba_cleaned.csv")

In [3]:
dtf_new = dtf_new.rename(columns={'label': 'labels', 'text_clean': 'text'})

In [4]:
dtf_new

,Unnamed: 0,bbc_url_id,date,news_title,labels,text
0,0,49561007,3 Oṣù Owewe 2019,Xenophobic Attack: Awọn ọmọ Nàìjíria tí yarí p...,world,xenophobic attack awọn ọmọ nàìjíria yarí yóò g...
1,1,47683813,24 Oṣù Ẹrẹ̀nà 2019,Árẹ̀wá: Bákan náà ni a kò fẹ́ Atiku torí dúkìá...,politics,árẹwá bákan fẹ atiku torí dúkìá àjọni tó fẹ tà
2,2,47375620,27 Oṣù Èrèlè 2019,2019 Election update: Buhari borí; ó di ààrẹ N...,politics,election update buhari borí di ààrẹ nàìjíríà t...
3,3,51350982,2 Oṣù Èrèlè 2020,Nathaniel Samuel: Ìwádìí tí bẹ̀rẹ̀ lórí afuras...,politics,nathaniel samuel ìwádìí bẹrẹ lórí afurasí tó g...
4,4,50975898,4 Oṣù Ṣẹ́rẹ́ 2020,Yoruba Film: Ṣé ẹ máa wá ní Pasuma ni mo bí àw...,entertainment,yoruba film ẹ pasuma ọmọ tó kuti jaiye
...,...,...,...,...,...,...
1903,184,49136947,27 Oṣù Agẹmọ 2019,Shiite Movement: Shiite fèsì lórí àṣẹ ilé ẹjọ́...,politics,shiite movement shiite fèsì lórí àṣẹ ilé ẹjọ ẹ...
1904,185,48594443,11 Oṣù Òkúdu 2019,9th Assembly: PDP ní láìpẹ́ làwọn aráàlú yóò m...,politics,9th assembly pdp láìpẹ làwọn aráàlú yóò mọ ìyà...
1905,186,46908855,17 Oṣù Ṣẹ́rẹ́ 2019,Adekunle Gold: Ọdún márùn ún ni mo fi mọ Simi ...,entertainment,adekunle gold ọdún márùn ún fi mọ simi ká tó ṣ...
1906,187,51094629,14 Oṣù Ṣẹ́rẹ́ 2020,Biafra at 50: Ọ̀nà tí ẹja panla gbà di gbajúgb...,politics,biafra at ọnà ẹja panla gbà di gbajúgbajà nínú...


In [5]:
data = dtf_new.drop(["date", "bbc_url_id", "Unnamed: 0", "news_title"], axis=1)

In [6]:
data

,labels,text
0,world,xenophobic attack awọn ọmọ nàìjíria yarí yóò g...
1,politics,árẹwá bákan fẹ atiku torí dúkìá àjọni tó fẹ tà
2,politics,election update buhari borí di ààrẹ nàìjíríà t...
3,politics,nathaniel samuel ìwádìí bẹrẹ lórí afurasí tó g...
4,entertainment,yoruba film ẹ pasuma ọmọ tó kuti jaiye
...,...,...
1903,politics,shiite movement shiite fèsì lórí àṣẹ ilé ẹjọ ẹ...
1904,politics,9th assembly pdp láìpẹ làwọn aráàlú yóò mọ ìyà...
1905,entertainment,adekunle gold ọdún márùn ún fi mọ simi ká tó ṣ...
1906,politics,biafra at ọnà ẹja panla gbà di gbajúgbajà nínú...


In [7]:
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()
data["labels"] = lb.fit_transform(dtf_new["labels"])

In [8]:
data

,labels,text
0,4,xenophobic attack awọn ọmọ nàìjíria yarí yóò g...
1,2,árẹwá bákan fẹ atiku torí dúkìá àjọni tó fẹ tà
2,2,election update buhari borí di ààrẹ nàìjíríà t...
3,2,nathaniel samuel ìwádìí bẹrẹ lórí afurasí tó g...
4,0,yoruba film ẹ pasuma ọmọ tó kuti jaiye
...,...,...
1903,2,shiite movement shiite fèsì lórí àṣẹ ilé ẹjọ ẹ...
1904,2,9th assembly pdp láìpẹ làwọn aráàlú yóò mọ ìyà...
1905,0,adekunle gold ọdún márùn ún fi mọ simi ká tó ṣ...
1906,2,biafra at ọnà ẹja panla gbà di gbajúgbajà nínú...


In [9]:
lb.classes_

array(['entertainment', 'health', 'politics', 'sport', 'world'],
      dtype=object)

In [10]:
data['labels']

0       4
1       2
2       2
3       2
4       0
       ..
1903    2
1904    2
1905    0
1906    2
1907    2
Name: labels, Length: 1908, dtype: int32

In [11]:
X = data

In [12]:
X_train, X_test = model_selection.train_test_split(X, test_size = 0.50, random_state = 12)
print(X_train.shape)
print(X_test.shape)

(954, 2)
(954, 2)


In [13]:
import pandas as pd

from simpletransformers.classification import ClassificationModel

# Train and Evaluation data needs to be in a Pandas Dataframe containing at least two columns. 
#If the Dataframe has a header, it should contain a 'text' and a 'labels' column. 
#If no header is present, the Dataframe should contain at least two columns, 
#with the first column is the text with type str, and the second column in the label with type int.
train_data = [
    ["Example sentence belonging to class 1", 1],
    ["Example sentence belonging to class 0", 0],
    ["Example eval senntence belonging to class 2", 2],
]
train_df = pd.DataFrame(train_data)

eval_data = [
    ["Example eval sentence belonging to class 1", 1],
    ["Example eval sentence belonging to class 0", 0],
    ["Example eval senntence belonging to class 2", 2],
]
eval_df = pd.DataFrame(eval_data)

# Create a ClassificationModel
#model = ClassificationModel(
 #   "bert",
  #  "bert-base-cased",
  #  num_labels=5,
  #  args={"reprocess_input_data": True, "overwrite_output_dir": True},
  #  use_cuda=False
    
#)

model = ClassificationModel(
    "bert",
    "bert-base-cased",
    num_labels=5,
    args={"reprocess_input_data": True, "overwrite_output_dir": True},
    use_cuda=False
    
)

# Train the model
model.train_model(X_train)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(X_test)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

C:\Users\Dee\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [14]:
import torch
torch.save(model, "model/bert_model_yoruba")

In [15]:
import torch
model = torch.load('model/bert_model_yoruba')

In [16]:
predictions, raw_outputs = model.predict(["Some arbitary sentence"])

In [17]:
raw_outputs

array([[ 0.09155511, -0.48879486,  1.01875508, -0.8865453 ,  0.13150549]])

In [21]:
model_outputs

array([[ 0.17233548, -0.59962738,  1.31216097, -0.94501436,  0.03234424],
       [ 0.16580307, -0.30738944,  1.21619701, -0.87827551,  0.029895  ],
       [ 0.27581298, -0.77012068,  1.40519524, -1.09593832,  0.02979188],
       ...,
       [ 0.17015244, -0.95915449,  1.75006032, -1.40330458, -0.14212191],
       [-0.03187372, -1.0282768 ,  1.88639092, -1.51467156, -0.28656393],
       [ 0.25801969, -1.07522094,  1.82083869, -1.51858115, -0.18404678]])

In [25]:
a = list(X_test['text'].iloc[0:20])
predictions, raw_outputs = model.predict(a)

In [26]:
a

['champion league athletico bomi paná ayọ juventus ìpadàbọ ronaldo ìlú madrid',
 'coronavirus àjọ tó rí ìwọléwọde sáàyè fẹnikẹni tó láárùn yí wọlé nàìjíríà',
 'bbc africa eye ìyá rántí ikú rẹ lásìkò ìrọbí àìbímọ tèmí pa mí kú díẹdíẹ',
 'nigeria election canada yá ọnà mì tọmọ nàìjíríá fi bínú sísún ìdìbò',
 'xenophobia ọsẹ tó bọ bàálù yóò kó ọmọ nàíjíríà wálé',
 'premier league liverpool to ọrẹ ẹyìn manchester united pẹlú àmì ayò méjì òdo',
 'adewale ayuba ogun ja àgbo fújì jà',
 'omoyele sowore fi ṣowore sílé tó pẹjù tàbí ṣewọn',
 'guru maharaji yẹ ìjọba àpapọ fíkún owó oṣù òṣìṣẹ adájọ',
 'lady jibowu ẹni ọdún ló dágbére fáyé dìgbóṣe',
 'hijab crisis fásitì ibadan ohun tó yípadà pẹlú ìwọṣọ akẹkọọ isi',
 'ibadan ìdàrúdàpọ wáyé láàrin ọlọjà yoruba igbo ipò lọjà ògùnpa',
 'xenophobia attack ilẹ south afrika gbàgbé nàìjíríà àtìlẹyìn',
 'tanker accident ọpọ èèyàn tó sẹnu iṣẹ okoòwò ló tàn sílẹ',
 'nlc ìpàdé ọjọ ajé yóò sọ bóyá á yansẹ lódì',
 'eidel kabir aisha buhari hajj pẹlú ìyàwó ààrẹ n

In [28]:
from sklearn.metrics import classification_report

print(classification_report(np.array(X_test["labels"].iloc[0:20]), predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         6
           1       0.00      0.00      0.00         1
           2       0.45      1.00      0.62         9
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00         2

    accuracy                           0.45        20
   macro avg       0.09      0.20      0.12        20
weighted avg       0.20      0.45      0.28        20



C:\Users\Dee\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [29]:
np.array(X_test["labels"])

array([3, 1, 0, 2, 4, 3, 0, 2, 0, 0, 2, 2, 4, 2, 2, 0, 2, 2, 2, 0, 2, 4,
       2, 4, 4, 2, 1, 2, 3, 0, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 4, 2, 4,
       4, 2, 2, 0, 2, 0, 1, 0, 2, 2, 2, 2, 2, 2, 4, 1, 2, 2, 2, 2, 0, 2,
       2, 2, 2, 2, 2, 4, 0, 1, 1, 2, 2, 2, 2, 4, 4, 4, 1, 0, 3, 4, 2, 0,
       2, 2, 2, 0, 2, 2, 2, 2, 4, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0, 4, 0, 2,
       0, 0, 2, 0, 2, 2, 3, 2, 2, 2, 3, 3, 2, 2, 0, 4, 3, 0, 0, 2, 2, 2,
       2, 2, 2, 3, 2, 2, 2, 0, 2, 2, 2, 2, 2, 4, 2, 2, 4, 0, 2, 2, 2, 2,
       2, 2, 0, 2, 2, 2, 2, 0, 0, 0, 2, 2, 2, 2, 0, 2, 2, 0, 0, 4, 2, 0,
       0, 0, 2, 0, 0, 2, 0, 0, 0, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1,
       2, 4, 2, 2, 0, 0, 3, 4, 2, 2, 2, 0, 0, 2, 0, 2, 2, 2, 2, 0, 2, 0,
       0, 3, 2, 2, 2, 2, 4, 2, 0, 4, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 0, 0,
       2, 2, 2, 0, 0, 0, 2, 2, 2, 2, 1, 2, 2, 0, 2, 0, 4, 4, 0, 0, 2, 4,
       2, 0, 4, 3, 2, 2, 4, 0, 2, 2, 2, 2, 2, 3, 2, 0, 2, 4, 2, 2, 0, 2,
       4, 1, 2, 0, 2, 2, 2, 0, 0, 2, 2, 4, 2, 2, 4,

In [30]:
predictions

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
      dtype=int64)

In [31]:
X_train

,labels,text
705,2,nkan tó jẹyọ níbi ìfọrọwánilẹnuwò fáwọn tó fẹ ...
1021,2,ibrahim elzakzaky ṣeésẹ elzakzaky òkè òkun ìtọ...
1125,0,sugar iléẹjọ afurasí mẹrin mọwọmẹsẹ lórí ikú s...
229,3,man utd v man city ṣèlẹrí alatilẹyin liverpool...
793,4,ìròyìn tó ọsẹ àkọkọ amẹríkà dojú ìjà kọ iran ì...
...,...,...
1265,4,togo election ààrẹ gnassingbe díje sáà kẹrin b...
1277,2,nigeria election nọmbà ló le fi iléèṣẹ ológun ...
1414,2,nan aráàlú se ló yẹ nan bẹbẹ òpin ìyansẹlódì asuu
1691,2,ekiti election appeal ilé ẹjọ kòtẹmilọrùn dá f...
